<a href="https://colab.research.google.com/github/facundocarballo/ProgramacionConcurrente/blob/main/TP3/Java.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sockets

Este sistema cuenta con 2 procesos, un proceso que hace de Servidor y otro proceso que hace de Cliente.

El cliente envia una cadena de caracteres (que puede contener numeros, signos y letras). El Servidor recibe esa informacion y la procesa. Obteniendo estos datos:

- Cantidad de caracteres totales.
- Cantidad de letras.
- Cantidad de digitos.
- Cantidad de caracteres que no son letra ni digito.

# Respuesta

In [26]:
%%writefile Respuesta.java
import java.io.Serializable;

public class Respuesta implements Serializable 
{
  private String caractTotales;
  private String cantLetras;
  private String cantDigitos;
  private String cantidadOtros;

  public Respuesta(String caractTotales, String cantLetras, String cantDigitos, String cantidadOtros)
  {
    this.caractTotales = caractTotales;
    this.cantLetras = cantLetras;
    this.cantDigitos = cantDigitos;
    this.cantidadOtros = cantidadOtros;
  }

  @Override
  public String toString()
  {
    return caractTotales + cantLetras + cantDigitos + cantidadOtros;
  }
  
  // Getter methods for private fields
  public String getCaractTotales()
  {
    return caractTotales;
  }

  public String getCantLetras()
  {
    return cantLetras;
  }

  public String getCantDigitos()
  {
    return cantDigitos;
  }

  public String getCantidadOtros()
  {
    return cantidadOtros;
  }
}


Overwriting Respuesta.java


# Servidor

In [25]:
%%writefile Servidor.java
import java.io.IOException;
import java.io.ObjectOutputStream;
import java.io.Serializable;
import java.net.ServerSocket;
import java.net.Socket;
import java.util.Scanner;

public class Servidor 
{
    private static final int PUERTO = 5000;

    public static void main(String[] args) 
    {
        try 
        {
            ServerSocket socketServidor = new ServerSocket(PUERTO);
            System.out.println("Servidor iniciado en el puerto " + PUERTO);

            System.out.println("Esperando conexión de un cliente...");
            Socket socketCliente = socketServidor.accept();
            System.out.println("Cliente conectado desde la dirección " + socketCliente.getInetAddress());

            Scanner entrada = new Scanner(socketCliente.getInputStream());
            ObjectOutputStream salida = new ObjectOutputStream(socketCliente.getOutputStream());

            String mensajeCliente = entrada.nextLine();
            System.out.println(mensajeCliente);

            int caracteresTotales = mensajeCliente.length();
            int cantidadLetras = 0;
            int cantidadDigitos = 0;
            int cantidadOtros = 0;

            for (char c : mensajeCliente.toCharArray()) 
            {
                if (Character.isLetter(c)) 
                {
                    cantidadLetras++;
                } 
                else if (Character.isDigit(c)) 
                {
                    cantidadDigitos++;
                } 
                else 
                {
                    cantidadOtros++;
                }
            }

            String s1 = "Cantidad de caracteres totales: " + caracteresTotales + "\n";
            String s2 = "Cantidad de letras: " + cantidadLetras + "\n";
            String s3 = "Cantidad de dígitos: " + cantidadDigitos + "\n";
            String s4 = "Cantidad de caracteres que no son letra ni dígito: " + cantidadOtros + "\n";
            Respuesta respuesta = new Respuesta(s1, s2, s3, s4);
            salida.writeObject(respuesta);

            entrada.close();
            salida.close();
            socketCliente.close();
            socketServidor.close();
        } catch (IOException e) 
        {
            e.printStackTrace();
        }
    }
}

Overwriting Servidor.java


# Cliente

In [24]:
%%writefile Cliente.java
import java.io.IOException;
import java.io.ObjectInputStream;
import java.net.Socket;

public class Cliente 
{
    private static final String DIRECCION_SERVIDOR = "localhost";
    private static final int PUERTO_SERVIDOR = 5000;

    public static void main(String[] args) throws ClassNotFoundException 
    {
        String mensaje = args[0] + "\n";

        try 
        {
            Socket socketCliente = new Socket(DIRECCION_SERVIDOR, PUERTO_SERVIDOR);
            socketCliente.getOutputStream().write(mensaje.getBytes());

            ObjectInputStream entrada = new ObjectInputStream(socketCliente.getInputStream());
            Respuesta respuesta = (Respuesta) entrada.readObject();
            System.out.println(respuesta);

            entrada.close();
            socketCliente.close();
        } catch (IOException e) 
        {
            e.printStackTrace();
        }
    }
}


Overwriting Cliente.java


# Ejecucion

Compilamos el Servidor, generando un archivo llamado Servidor.class

In [4]:
!javac Servidor.java 

Compilamos el Cliente, generando un archivo llamado Cliente.class

In [5]:
!javac Cliente.java 

Compilamos la Respuesta, generando un archivo llamado Respuesta.class

In [6]:
!javac Respuesta.java 

Ejecutamos en segundo plano el Servidor.

In [17]:
!nohup java Servidor > Servidor.txt 2> Servidor.txt &

Visualizamos que este corriendo el Servidor.

In [18]:
!ps -ef | grep Servidor.class

root        3360     593  0 19:25 ?        00:00:00 /bin/bash -c ps -ef | grep Servidor.class
root        3362    3360  0 19:25 ?        00:00:00 grep Servidor.class


Verificamos que no haya imprimido ningun error el Servidor a la hora de crearse.

In [19]:
!cat Servidor.txt

Servidor iniciado en el puerto 5000
Esperando conexión de un cliente...


Ejecutamos en segundo plano al Cliente.

In [22]:
!nohup java Cliente hola1234 > Cliente.txt 2> Cliente.txt &

Visualizamos la salida del archivo Cliente.

In [23]:
!cat Cliente.txt

Cantidad de caracteres totales: 8
Cantidad de letras: 4
Cantidad de dígitos: 4
Cantidad de caracteres que no son letra ni dígito: 0

